In [126]:
import pandas as pd
import numpy as np
import seaborn as sns
from pulp import *
import warnings
warnings.filterwarnings('ignore')
df=pd.read_csv('preview.csv')
new_df = df[['league','home_team', 'away_team']]

In [127]:
def check_result(row):
    game=(df['home_team']==row['home_team']) & (df['away_team']==row['away_team'])
    proba=list(df[game][['Home','Draw','Away']].values[0])
    max_proba=proba.index(max(proba))
    if max_proba == 0:
        result=list(df[game][['Home','odds_home']].values[0])
        result.append('Home')
    elif max_proba == 1:
        result = list(df[game][['Draw','odds_draw']].values[0])
        result.append('Draw')
    else:
        result = list(df[game][['Away','odds_away']].values[0])
        result.append('Away')
    return result

In [128]:
new_df[['proba','odds','aposta']]=new_df.apply(lambda row: check_result(row), axis=1,result_type="expand")
new_df.reset_index(drop=True,inplace=True)
new_df['ev']=new_df['odds']*new_df['proba']
new_df=new_df.loc[new_df['proba']>=0.75,:].reset_index(drop=True)

In [144]:
valor=100
home_teams=list(new_df['home_team'])
odds=dict(zip(home_teams,list(new_df['odds'])))
evs=dict(zip(home_teams,list(new_df['ev'])))
my_vars={}
vars_conservador={}
for i in range(0,len(home_teams)):
    my_vars[home_teams[i]]=LpVariable(home_teams[i],0,0.15,cat='Float')
for game in my_vars.keys():
    if odds[game] < 1.5:
        vars_conservador[game]=odds[game]

In [168]:
prob = LpProblem("Calc weights", LpMaximize)
prob += lpSum([my_vars[game]*evs[game] for game in my_vars.keys()])
prob += lpSum([my_vars[game] for game in my_vars.keys()]) == 1
prob += lpSum([my_vars[game]*evs[game]*valor for game in vars_conservador.keys()]) >= valor*0.7
for game in my_vars.keys():
    prob += my_vars[game]*valor >= 1
    prob += my_vars[game]*valor <= 10
solver = CPLEX_PY()
status = prob.solve(solver)
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)

PulpSolverError: CPLEX_PY: Not Available

In [8]:

"""
The Beer Distribution Problem for the PuLP Modeller
Authors: Antony Phillips, Dr Stuart Mitchell  2007
"""

# Import PuLP modeler functions
from pulp import *

# Creates a list of all the supply nodes
Warehouses = ["A", "B"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 1000,
          "B": 4000}

# Creates a list of all demand nodes
Bars = ["1", "2", "3", "4", "5"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {"1":500,
          "2":900,
          "3":1800,
          "4":200,
          "5":700,}

# Creates a list of costs of each transportation path
costs = [   #Bars
         #1 2 3 4 5
         [2,4,5,2,1],#A   Warehouses
         [3,1,3,2,3] #B
         ]

# The cost data is made into a dictionary
costs = makeDict([Warehouses,Bars],costs,0)

# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Beer Distribution Problem",LpMinimize)

# Creates a list of tuples containing all the possible routes for transport
Routes = [(w,b) for w in Warehouses for b in Bars]

# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route",(Warehouses,Bars),0,None,LpInteger)

# The objective function is added to 'prob' first
prob += lpSum([vars[w][b]*costs[w][b] for (w,b) in Routes]), "Sum_of_Transporting_Costs"

# The supply maximum constraints are added to prob for each supply node (warehouse)
for w in Warehouses:
    prob += lpSum([vars[w][b] for b in Bars])<=supply[w], "Sum_of_Products_out_of_Warehouse_%s"%w

# The demand minimum constraints are added to prob for each demand node (bar)
for b in Bars:
    prob += lpSum([vars[w][b] for w in Warehouses])>=demand[b], "Sum_of_Products_into_Bar%s"%b
                   
# The problem data is written to an .lp file
prob.writeLP("BeerDistributionProblem.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen    
print("Total Cost of Transportation = ", value(prob.objective))

Status: Optimal
Route_A_1 = 300.0
Route_A_2 = 0.0
Route_A_3 = 0.0
Route_A_4 = 0.0
Route_A_5 = 700.0
Route_B_1 = 200.0
Route_B_2 = 900.0
Route_B_3 = 1800.0
Route_B_4 = 200.0
Route_B_5 = 0.0
Total Cost of Transportation =  8600.0
